# Volunteering Data

Data extracted from Rosterfy and processes below.

In [76]:
import pandas as pd
from pathlib import Path
import os
from datetime import datetime as dt

ROOT = Path('../')
ROOT.resolve()

PosixPath('/Users/tazminchiles/Documents/GitHub/bradford-2025')

### Process summaries

In [77]:
WORKING_DIR = os.path.join(ROOT, 'src', 'volunteers', '_data')
OUT_DIR = os.path.join(ROOT, 'src', 'volunteers', '_data')

checkpoints_monthly = pd.read_csv(os.path.join(WORKING_DIR, 'checkpoints_monthly.csv'))
geo_volunteer = pd.read_csv(os.path.join(WORKING_DIR, 'geo_summary.csv'))
shifts_monthly = pd.read_csv(os.path.join(WORKING_DIR, 'shifts_monthly.csv'))
shifts_weekly = pd.read_csv(os.path.join(WORKING_DIR, 'shifts_weekly.csv'))

Convert timestamp columns to datetime to work with OI Lume Viz

In [81]:
checkpoints_monthly['Month ending'] = pd.to_datetime(checkpoints_monthly['Month ending']).dt.strftime('%Y-%m-%d')
shifts_monthly['Month ending'] = pd.to_datetime(shifts_monthly['month ending']).dt.strftime('%Y-%m-%d')
shifts_weekly['Month ending'] = pd.to_datetime(shifts_weekly['week ending']).dt.strftime('%Y-%m-%d')

checkpoints_monthly.to_csv(os.path.join(OUT_DIR, 'checkpoints_monthly.csv'), index=False)
shifts_monthly.to_csv(os.path.join(OUT_DIR, 'shifts_monthly.csv'), index=False)
shifts_weekly.to_csv(os.path.join(OUT_DIR, 'shifts_weekly.csv'), index=False)

## Exploring volunteer demographics

In [82]:
#TODO